# Monitoring and Muxing

## Configuration

This code would normally go in a script automatically run at startup. The user would not have to worry about this.

In [0]:
%matplotlib notebook
import matplotlib.pyplot as plt

# Make plots live-update while scans run.
from bluesky.utils import install_nb_kicker
install_nb_kicker()

from ophyd import Signal
import asyncio
import itertools
from bluesky import RunEngine
import bluesky.plans as bp
from bluesky.callbacks import CallbackBase
from bluesky.examples import SynGauss, det1, det2
import datetime

RE = RunEngine()

# Set up a databroker.
import os
import tzlocal
import warnings
from portable_mds.hdf5.mds import MDS
from portable_fs.sqlite.fs import FileStore
import tempfile


dirname = tempfile.TemporaryDirectory().name
mds = MDS({'directory': dirname,
           'timezone': tzlocal.get_localzone().zone})
fs = FileStore({'dbpath': os.path.join(dirname, 'filestore.db')})

# Filter warnings to avoid confusing new users with red deprecation warnings.
warnings.simplefilter("ignore")

from databroker.broker import Broker
db = Broker(mds, fs)
RE.subscribe('all', db.insert);  # all data generated by RE will be saved into db


def _iso_from_timestamp(ts):
    return datetime.datetime.fromtimestamp(ts).isoformat()


class TimePrinter(CallbackBase):
    def __init__(self):
        self.seen_descriptors = {}

    def start(self, doc):
        d = _iso_from_timestamp(doc['time'])
        print('{} scan {} [{}] started by {}'.format(
            d, doc['uid'][:6], doc['scan_id'], doc.get('user', 'unknown')))

    def descriptor(self, doc):
        self.seen_descriptors[doc['uid']] = doc

    def event(self, doc):
        d = self.seen_descriptors[doc['descriptor']]
        t = _iso_from_timestamp(doc['time'])
        print('{} event from {} stream'.format(t, d['name']))

    def stop(self, doc):
        t = _iso_from_timestamp(doc['time'])
        print('{} finished run {}'.format(t, doc['run_start'][:6]))

### Set up a signal whose value changes in the background
### so we can monitor it. All of the complexity is around faking
### up the signal -- with real hardware this is more straightforward.

def flipper_factory(signal, frequency, sequence, loop=None):
    if loop is None:
        loop = asyncio.get_event_loop()
    delay = 1 / frequency

    async def inner():

        for v in itertools.cycle(sequence):
            ev = asyncio.Event(loop=loop)
            st = signal.set(v)
            st.finished_cb = lambda : loop.call_soon_threadsafe(
                ev.set)
            await asyncio.sleep(delay)
            await ev.wait()

    return inner()

motor = Signal(name='motor', value=0)
det = SynGauss('det',
           motor, 'motor',
           center=1,
           Imax=1000,
           sigma=5,
           noise='poisson',
           exposure_time=0.1)

sig = Signal(name='flipper', value=0)
task = RE.loop.create_task(flipper_factory(sig, 5, [0, 1, 2]))

## Data Acquisition

### Monitor `sig` asynchronously while counting `det` five times.

In [0]:
def monitor_count(detectors, monitors, num=5, delay=1):
    "Monitor `monitors` while counting `detectors`."
    # Instantiate a normal 'count' plan.
    plan = bp.count(detectors, num=num, delay=delay)

    # Use a 'preprocessor', monitoring_during_wrapper, to add monitors.
    wrapped_plan = bp.monitor_during_wrapper(plan, monitors)
    return (yield from wrapped_plan)

The ``TimePrinter`` callback prints a line whenever we capture a reading from `det` or `sig`.

In [0]:
RE(monitor_count([det], [sig], delay=1), TimePrinter())

### Access the saved data

The data is organized into two "streams", which we can access as tables where each row is an "event" -- a group of readings taken at the same time.

In [0]:
header = db[-1]
header.table()  # shows the 'primary' stream by default

In [0]:
header.table(stream_name='primary')  # equivalent to the above

In [0]:
header.table(stream_name='flipper_monitor')

### Count two detectors synchronously while again monitoring `sig` asynchronously.

In [0]:
RE(monitor_count([det, det2], [sig], delay=1), TimePrinter())

In [0]:
header = db[-1]

The readings from `det` and `det2` are shown in the same table because their readings are synchronous.

In [0]:
header.table()

### Plot data stream together

In [0]:
fig, ax = plt.subplots()

ax.plot('time', 'det', data=header.table(), marker='o', label='det')
ax.plot('time', 'det2', data=header.table(), marker='o', label='det2')
ax.plot('time', 'flipper', data=header.table(stream_name='flipper_monitor'), marker='x', label='flipper')
ax.legend()

### Interpolate and normalize

In [0]:
# Fetch all streams at once.
header.table(stream_name=db.ALL)

In [0]:
# Make 'time' the index and sort on it.
header.table(stream_name=db.ALL).set_index('time').sort_index()

In [0]:
# Interpolate using 'forward filling'.
header.table(stream_name=db.ALL).set_index('time').sort_index().ffill()

In [0]:
data = header.table(stream_name=db.ALL).set_index('time').sort_index().ffill()
data['det'] * data['flipper']

In [0]:
plt.figure()
(data['det'] * data['flipper']).plot()

## Exercises

1. Execute the ``monitor_count`` with a different ``delay`` parameter to verify that the readings from ``sig`` come at 5 Hz.